
# Objection 工具
　　参考：[实用FRIDA进阶：内存漫游、hook anywhere、抓包](https://www.anquanke.com/post/id/197657#h3-6)  

## 一、简介
　　
  
## 二、安装
　　objection需要配合frida和frida-tools使用,且安装前置条件是：python版本 > 3.4 　　pip版本 > 9.0   
>pip install objection

## 二、使用

```shell
objection --help      # 查看帮助

help frida                # 不知道当前命令的作用 进入objection后就在命令前加 help 会有提示
objection -g 包名 explore     # 注入进程 objection没有找到进程 会以spwan方式启动进程
env 　　　　　　# 枚举与所讨论的应用程序相关的其他有趣目录(查看枚举应用包相关的一些文件目录)

android hooking list classes   # 列出内存中所有的类
android hooking search classes 包名   # 在内存中所有已加载的类中搜索包含特定关键词的类
android hooking list class_methods 包名.类名   # 列出类的所有方法
android hooking watch class 包名.类名     # hook类的所有方法
android hooking list activities  # 查看Activity，service相同
android intent launch_activity com.android.settings.DisplaySettings  # 实现Activity跳转
android heap search instances com.android.settings.DisplaySettings   # 搜索类的实例
android heap execute 0x2526 getPreferenceScreenResId    # 主动调用实例
android hooking search methods display  # 列出内存中所有的方法
android hooking watch class_method android.bluetooth.BluetoothDevice.getName --dump-args --dump-return --dump-backtrace  # 打印具体方法的参数、返回值、堆栈信息

jobs list         # 查看hook了多少个类
jobs kill jobid      # 把正在hook的任务关闭

# hook方法的参数、返回值和调用栈(–dump-args: 显示参数; --dump-return: 显示返回值; --dump-backtrace: 显示堆栈)
android hooking watch class_method 包名.类名.方法 --dump-args --dump-return --dump-backtrace

android hooking watch class_method 包名.类名.方法        # 默认会Hook方法的所有重载

# 如果只需hook其中一个重载函数 指定参数类型 多个参数用逗号分隔  
android hooking watch class_method 包名.类名.方法 "包名.类名.方法"       

android heap search instances 包名.类名 --fresh         # 搜索堆中的实例

android heap execute 地址(hashcode的地址) 方法名         # 调用实例的方法

memory list modules         # 枚举内存中所有模块

memory list exports 文件名.so           # 枚举模块中所有导出函数

objection -N -h 192.168.1.3 -p 9999 -g 包名 explore           # 指定ip和端口的连接

objection -N -h 192.168.1.3 -p 9999 -g 包名 explore --startup-command "android hooking watch class '包名.类名'"        # spawn启动前就Hook

objection -d -g 包名 explore --startup-command "android hooking watch class_method '包名.类名.方法名' --dump-backtrace --dump-args --dump-return"        # spawn启动前就Hook指定类方法，并打印参数、堆栈、返回值

# spawn启动前就Hook 打印参数、返回值、函数调用栈
objection -N -h 192.168.1.3 -p 9999 -g 包名 explore --startup-command "android hooking watch class_method '包名.类名.方法名'
```

### 2.2、Memory 指令
```shell
memory list modules  // 查看内存中加载的库
memory list exports libssl.so  // 查看库的导出函数
memory list exports libart.so --json /root/libart.json //将结果保存到json文件中
memory search --string --offsets-only //搜索内存
memory search "64 65 78 0a 30 35 00"
```
### 2.3、android heap
```shell
android heap search instances com.android.settings.DisplaySettings //堆内存中搜索指定类的实例, 可以获取该类的实例id
android heap execute 0x2526 getPreferenceScreenResId //直接调用指定实例下的方法
android heap evaluate 0x2526 //自定义frida脚本, 执行实例的方法
```
### 2.4、root 
```shell
android root disable     # 尝试关闭app的root检测
android root simulate　　　　　# 尝试模拟root环境
```
### 2.5、activities
```shell
android hooking list activities　　　　　# 可以列出app具有的所有avtivity
android intent launch_activity [class_activity]　　　　# 启动指定avtivity
android hooking list class_methods com.android.settings.DisplaySettings　　　　# 查看类的全部方法
```

### 2.6、UI相关
```shell
android ui screenshot [image.png]　　　　　# 截图
android ui FLAG_SECURE false　　　　　# 设置FLAG_SECURE权限
```

### 2.7、内存漫游
```shell
android hooking list classes　　　　# 列出内存中所有的类
android hooking search classes [search_name] 　　　　# 在内存中所有已加载的类中搜索包含特定关键词的类
android hooking search methods [search_name] 　　　　# 在内存中所有已加载的方法中搜索包含特定关键词的方法
android hooking generate simple [class_name]　　　　# 直接生成hook代码
```
### 2.8、hook 方式
　　hook指定方法, 如果有重载会hook所有重载,且可以通过下面命令参数查看方法信息。
  + --dump-args : 打印参数
  + --dump-backtrace : 打印调用栈
  + --dump-return : 打印返回值
  
```shell
android hooking watch class_method com.xxx.xxx.methodName --dump-args --dump-backtrace --dump-return
android hooking watch class_method java.lang.StringBuilder.toString --dump-return //获取全部tostring的返回来值
android hooking watch class_method android.app.Dialog.show --dump-args --dump-backtrace --dump-return //弹窗
android hooking watch class com.xxx.xxx 　　　　　# hook指定类, 会打印该类下的所以调用 
android hooking set return_value com.xxx.xxx.methodName false　　　　　# 设置返回值(只支持bool类型) 
```

### 2.9、Spawn方式Hook
```shell
objection -g packageName explore --startup-command '[obejection_command]'     # 启东时hook
```

### 2.10、activity和service操作
```shell
android hooking list activities　　　　# 枚举activity
android intent launch_activity [activity_class]　　　　# 启动activity
android hooking list services　　　　　# 枚举services
android intent launch_service [services_class]　　　　# 启动services
```

### 2.11、任务管理器
```shell
jobs list　　　　# 查看任务列表
jobs kill [task_id]　　　　　# 关闭任务
```
### 2.12、关闭app的ssl校验
```shell
android sslpinning disable
```
### 2.13、监控系统剪贴板
```shell
help android  clipboard　　　　　# 获取Android剪贴板服务上的句柄并每5秒轮询一次用于数据。 如果发现新数据，与之前的调查不同，则该数据将被转储到屏幕上。
```

### 2.14、执行命令行
```shell
help android shell_exec [command]
``` 

 ### Spawn方式Hook
 　　从Objection的使用操作中我们可以发现，Obejction采用Attach附加模式进行Hook，这可能会让我们错过较早的Hook时机，可以通过如下的代码启动Objection，引号中的objection命令会在启动时就注入App。   
>objection -g packageName explore --startup-command 'android hooking watch xxx'    


